# Data processing for census sections, incomes, population and buildings

In the below data processing pipeline, the following datasets are used and generated:
1. **seccionado.geojson**: Data about census sections, downloaded from INE
2. **seccionado_procesado_EPSG_25830.geojson**: Subset of the previous dataset, containing only three fields ('CUSEC', 'area_m2', 'geometry')
3. **renta.csv**: Income dataset downloaded from INE
4. **renta_procesado_2020.csv**: Income by census section
5. **poblacion.csv**: Population dataset downloaded from INE
6. **poblacion_procesado_2020.csv**: Population by section
7. **renta_poblacion_secciones_2020.csv**: Income and Population for each section
8. **edificios_adecuados_EPSG_25830.geojson**: Buildings from Crevillent with Solar radiation data
9. **edificios_secciones_crevillente**: Sections, buildings and radiation data. This is the final dataset for querying in the backend.

**Note**: For building a final dataset for other region, solar radiation data for that area is necessary.


## Census sections
Data from 2023. Downloaded from: https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout

In [1]:
import dask_geopandas as dask_gpd
dgdf = dask_gpd.read_file('seccionado.geojson', npartitions=4) #EPSG:25830 (m2)

In [2]:
dgdf['area_m2'] = dgdf.area
dgdf = dgdf[['CUSEC','NMUN','area_m2', 'geometry']]

In [3]:
computed_gdf = dgdf.compute()

In [4]:
computed_gdf.to_file('seccionado_procesado_EPSG_25830.geojson', driver='GeoJSON')

## Income by census sections
Data downloaded from: https://www.ine.es/dynt3/inebase/index.htm?padre=7132.

In [15]:
import dask.dataframe as dd
dtype_dict = {
    'Distritos': 'object',
    'Secciones': 'object',
    'Total': 'object'
}

In [16]:
df = dd.read_csv('renta.csv', dtype=dtype_dict, sep=';')

In [17]:
df = df[['Secciones', 'Indicadores de renta media', 'Periodo', 'Total']]

In [18]:
df = df[df['Indicadores de renta media'] == 'Renta neta media por hogar']

In [19]:
df = df[df['Periodo'] == 2020]

In [20]:
df = df[['Secciones', 'Periodo', 'Total']]

In [21]:
df = df.dropna()

In [22]:
df = df[['Secciones', 'Total']] # Periodo = 2020

In [23]:
df['Secciones'] = df['Secciones'].str.split().str[0]

In [24]:
df.to_csv('renta_procesado_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\2.part']

In [25]:
filenames = "renta_procesado_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
    'Total': 'object'
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',')

In [26]:
ddf.compute().to_csv('renta_procesado_2020.csv', index=False)

## Population

Data downloaded from: https://www.ine.es/jaxiT3/Tabla.htm?t=30868&L=0

In [27]:
import dask.dataframe as dd
dtype_dict = {
    'Secciones': 'object',
    'Indicadores demográficos': 'object'

}
df = dd.read_csv('poblacion.csv', dtype=dtype_dict, sep=';')

In [28]:
df = df[df['Periodo'] == 2020]

In [29]:
df = df[['Secciones', 'Indicadores demográficos', 'Total']]

In [30]:
df['Secciones'] = df['Secciones'].str.split().str[0]

In [31]:
df['Indicadores demográficos'] = df['Indicadores demográficos'].astype('category')

In [32]:
df['Indicadores demográficos'] = df['Indicadores demográficos'].cat.as_known()

In [33]:
df = df.dropna()

In [34]:
df_edad_media = df[df['Indicadores demográficos'] == 'Edad media de la población']
df_poblacion = df[df['Indicadores demográficos'] == 'Población']
df_porcentaje_unipersonales = df[df['Indicadores demográficos'] == 'Porcentaje de hogares unipersonales']
df_porcentaje_mayor_65 = df[df['Indicadores demográficos'] == 'Porcentaje de población de 65 y más años']
df_porcentaje_menor_18 = df[df['Indicadores demográficos'] == 'Porcentaje de población menor de 18 años']
df_porcentaje_poblacion = df[df['Indicadores demográficos'] == 'Porcentaje de población española']
df_tamaño_medio = df[df['Indicadores demográficos'] == 'Tamaño medio del hogar']

In [35]:
df_dataset = dd.merge(df_edad_media, df_poblacion, on='Secciones', how='inner', suffixes=('_edad_media', '_poblacion'))
df_dataset = df_dataset[['Secciones', 'Total_edad_media', 'Total_poblacion']]
df_dataset = dd.merge(df_dataset, df_porcentaje_unipersonales, on='Secciones', how='inner', suffixes=('_%_unipersonales', '_%_unipersonales'))
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_Hogares_unipersonales'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_mayor_65, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_mayor_65'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_menor_18, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_menor_18'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_poblacion, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_poblacion'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_tamaño_medio, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Tamaño_medio_hogar'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])

In [36]:
df_dataset.to_csv('poblacion_procesado_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\2.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\3.part']

In [37]:
filenames = "poblacion_procesado_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',')

In [38]:
ddf.compute().to_csv('poblacion_procesado_2020.csv', index=False)

## Data fusion: population, income and sections

In [7]:
import dask.dataframe as dd
dtype_dict = {
    'Secciones': 'object',
    'Total': 'object'

}

df_renta = dd.read_csv('renta_procesado_2020.csv', dtype=dtype_dict, sep=',')

In [8]:
df_renta = df_renta.rename(columns={"Total":"Renta_media"})

dtype_dict = {
    'Secciones': 'object',

}

In [9]:
df_poblacion = dd.read_csv('poblacion_procesado_2020.csv', dtype=dtype_dict, sep=',')

In [10]:
df_renta_poblacion = df_renta.merge(df_poblacion, on='Secciones', how='inner').compute()

### Joining with sections

In [5]:
import dask_geopandas as dg
df_seccionado = dg.read_file('seccionado_procesado_EPSG_25830.geojson', npartitions=4)
df_seccionado = df_seccionado.rename(columns={"CUSEC":"Secciones", "NMUN":"Municipios"})


In [11]:
import geopandas as gpd
# df_renta_poblacion_seccion = df_renta_poblacion.merge(df_seccionado, on='Secciones', how='inner')
df_renta_poblacion_seccion = gpd.GeoDataFrame(df_renta_poblacion).merge(df_seccionado.compute(), on='Secciones', how='inner')


In [12]:
dd_renta_poblacion_seccion = dd.from_pandas(df_renta_poblacion_seccion, npartitions=4)

In [13]:
dd_renta_poblacion_seccion.to_csv('renta_poblacion_secciones_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\2.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\3.part']

In [14]:
filenames = "renta_poblacion_secciones_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
    'Renta_media': 'object',
    'Total_edad' : 'object',
    'area_m2': 'object'
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',', thousands='.')

In [15]:
%who

computed_gdf	 dask_gpd	 dd	 dd_renta_poblacion_seccion	 ddf	 df_poblacion	 df_renta	 df_renta_poblacion	 df_renta_poblacion_seccion	 
df_seccionado	 dg	 dgdf	 dtype_dict	 filenames	 gpd	 


In [16]:
# To free RAM memory
import gc
del computed_gdf
del dask_gpd
del df_dataset
del df_edad_media
del df_poblacion
del df_porcentaje_mayor_65
del df_porcentaje_menor_18
del df_porcentaje_poblacion
del df_porcentaje_unipersonales
del df_renta
del df_renta_poblacion
del df_renta_poblacion_seccion
del df_seccionado
del df_tamaño_medio
gc.collect()

NameError: name 'df_dataset' is not defined

In [17]:
ddf.compute().to_csv('renta_poblacion_secciones_2020.csv', index=False)

## Data fusion: Solar cadaster

In [18]:
%who

dd	 dd_renta_poblacion_seccion	 ddf	 df_poblacion	 df_renta	 df_renta_poblacion	 df_renta_poblacion_seccion	 df_seccionado	 dg	 
dgdf	 dtype_dict	 filenames	 gc	 gpd	 


In [19]:
del dd_renta_poblacion_seccion
del ddf
del df
del dgdf
gc.collect()

NameError: name 'df' is not defined

In [1]:
import pandas as pd
df_secciones = pd.read_csv('renta_poblacion_secciones_2020.csv')

In [2]:
import geopandas as gpd
from shapely import wkt

df_secciones['geometry'] = df_secciones['geometry'].apply(wkt.loads)
gdf_secciones = gpd.GeoDataFrame(df_secciones, geometry='geometry')
gdf_secciones.crs = 'EPSG:25830'

To load only data from **Crevillent cadaster**:

In [3]:
gdf_edificios_adecuados = gpd.read_file('edificios_adecuados_EPSG_25830.geojson')
gdf_edificios_adecuados = gdf_edificios_adecuados[['reference', 'informatio', 'currentUse', 'AREA', 'MEAN','MWh_aprove', 'MWh_prod_e', 'geometry' ]]

C:\Users\omar.alvarez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\geopandas\io\file.py:383: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore")
C:\Users\omar.alvarez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\geopandas\io\file.py:387: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


In [4]:
gdf_edificios_adecuados['centroid'] = gdf_edificios_adecuados.geometry.centroid
gdf_edificios_adecuados = gdf_edificios_adecuados.set_geometry('centroid')

In [5]:
gdf_edificios_secciones = gpd.sjoin(gdf_edificios_adecuados, gdf_secciones, how="inner", predicate="within")
gdf_edificios_secciones = gdf_edificios_secciones.drop('index_right', axis=1)

In [6]:
secciones_no_crevillent = [300502002, 305903003, 390401001, 306507023]
gdf_edificios_secciones = gdf_edificios_secciones[~gdf_edificios_secciones['Secciones'].isin(secciones_no_crevillent)]

In [7]:
gdf_edificios_secciones['currentUse'] = gdf_edificios_secciones['currentUse'].str.split('_').str.get(-1)

In [8]:
# Procesar columnas string a numérirco
gdf_edificios_secciones['Total_edad_media'] = gdf_edificios_secciones['Total_edad_media'].str.replace(',', '.').astype(float)
gdf_edificios_secciones['Porcentaje_Hogares_unipersonales'] = gdf_edificios_secciones['Porcentaje_Hogares_unipersonales'].str.replace(',', '.').astype(float)
gdf_edificios_secciones['Porcentaje_mayor_65'] = gdf_edificios_secciones['Porcentaje_mayor_65'].str.replace(',', '.').astype(float)
gdf_edificios_secciones['Porcentaje_menor_18'] = gdf_edificios_secciones['Porcentaje_menor_18'].str.replace(',', '.').astype(float)
gdf_edificios_secciones['Porcentaje_poblacion'] = gdf_edificios_secciones['Porcentaje_poblacion'].str.replace(',', '.').astype(float)
gdf_edificios_secciones['Tamaño_medio_hogar'] = gdf_edificios_secciones['Tamaño_medio_hogar'].str.replace(',', '.').astype(float)

In [9]:
gdf_edificios_secciones.to_csv('edificios_secciones_crevillente.csv', index=False)

In [10]:
gdf_edificios_secciones

,reference,informatio,currentUse,AREA,MEAN,MWh_aprove,MWh_prod_e,geometry,centroid,Secciones,Renta_media,Total_edad_media,Total_poblacion,Porcentaje_Hogares_unipersonales,Porcentaje_mayor_65,Porcentaje_menor_18,Porcentaje_poblacion,Tamaño_medio_hogar,Municipios,area_m2
0,000100100XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,286.25,1487.646565,425.838829,76.650989,"MULTIPOLYGON (((690850.720 4229633.520, 690849...",POINT (690846.872 4229628.685),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
6,000100200XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,318.75,1377.799223,439.173502,79.051230,"MULTIPOLYGON (((690450.583 4229439.105, 690451...",POINT (690460.719 4229449.232),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
23,000100600XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,429.75,1466.006952,630.016488,113.402968,"MULTIPOLYGON (((690403.571 4229635.267, 690407...",POINT (690394.945 4229627.186),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
109,000200100XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,45.00,1456.424836,65.539118,11.797041,"MULTIPOLYGON (((691877.345 4226831.294, 691869...",POINT (691872.185 4226833.860),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
113,000200200XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,113.25,1436.972393,162.737124,29.292682,"MULTIPOLYGON (((691878.565 4226850.104, 691880...",POINT (691877.658 4226842.625),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7563,2457114XH9325N,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,214.00,1428.982839,305.802328,55.044419,"MULTIPOLYGON (((692299.048 4235492.607, 692300...",POINT (692304.769 4235483.061),305901008,22.028,37.6,1899,21.0,10.3,22.9,89.6,2.81,Crevillent,4.570897e+04
7570,2555603XH9325N,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,651.75,1395.759978,909.686565,163.743582,"MULTIPOLYGON (((692409.789 4235284.222, 692409...",POINT (692428.934 4235298.492),305901008,22.028,37.6,1899,21.0,10.3,22.9,89.6,2.81,Crevillent,4.570897e+04
7571,2555604XH9325N,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,666.50,1372.231518,914.592307,164.626615,"MULTIPOLYGON (((692448.526 4235323.171, 692448...",POINT (692465.225 4235321.974),305901008,22.028,37.6,1899,21.0,10.3,22.9,89.6,2.81,Crevillent,4.570897e+04
7572,2555605XH9325N,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,690.75,1384.114912,956.077375,172.093928,"MULTIPOLYGON (((692476.810 4235349.343, 692476...",POINT (692492.126 4235353.991),305901008,22.028,37.6,1899,21.0,10.3,22.9,89.6,2.81,Crevillent,4.570897e+04


## Convert coordinates from 'EPSG:25830' to 'EPSG:4326', with two new attributes: 'latitude' and 'longitude'

In [11]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from shapely.wkt import loads

In [12]:
buildings_sections_df = pd.read_csv('edificios_secciones_crevillente.csv')
buildings_sections_df.head()

,reference,informatio,currentUse,AREA,MEAN,MWh_aprove,MWh_prod_e,geometry,centroid,Secciones,Renta_media,Total_edad_media,Total_poblacion,Porcentaje_Hogares_unipersonales,Porcentaje_mayor_65,Porcentaje_menor_18,Porcentaje_poblacion,Tamaño_medio_hogar,Municipios,area_m2
0,000100100XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,286.25,1487.646565,425.838829,76.650989,"MULTIPOLYGON (((690850.7199999997 4229633.52, ...",POINT (690846.8720632484 4229628.684521084),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
1,000100200XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,318.75,1377.799223,439.173502,79.051230,MULTIPOLYGON (((690450.5832000002 4229439.1045...,POINT (690460.718536559 4229449.231857384),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
2,000100600XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,429.75,1466.006952,630.016488,113.402968,MULTIPOLYGON (((690403.5710000005 4229635.2670...,POINT (690394.9452679814 4229627.186024805),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
3,000200100XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,45.00,1456.424836,65.539118,11.797041,MULTIPOLYGON (((691877.3453000002 4226831.2942...,POINT (691872.1849999106 4226833.860002843),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07
4,000200200XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,113.25,1436.972393,162.737124,29.292682,MULTIPOLYGON (((691878.5654999996 4226850.1044...,POINT (691877.6579707734 4226842.624944544),305905002,21.927,44.9,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07


In [13]:
buildings_sections_df['geometry'] = buildings_sections_df['geometry'].apply(wkt.loads)
buildings_sections_gdf = gpd.GeoDataFrame(buildings_sections_df, geometry='geometry', crs="EPSG:25830")

In [14]:
from shapely.wkt import loads

# Convertir la columna 'centroid' en geometrías usando 'loads' de Shapely
buildings_sections_gdf['centroid'] = buildings_sections_gdf['centroid'].apply(lambda x: loads(str(x)))

# Ahora, convierte la columna 'centroid' en una GeoSeries
centroid_geo = gpd.GeoSeries(buildings_sections_gdf['centroid'], crs="EPSG:25830")

# Cambia la proyección de la GeoSeries
centroid_geo_crs = centroid_geo.to_crs(epsg=4326)

# Reemplaza la columna original con la transformada
buildings_sections_gdf['centroid'] = centroid_geo_crs


In [15]:
buildings_sections_gdf['centroid'] = buildings_sections_gdf['centroid'].to_crs(epsg=4326)

In [16]:
buildings_sections_gdf['longitude'] = buildings_sections_gdf['centroid'].x
buildings_sections_gdf['latitude'] = buildings_sections_gdf['centroid'].y

In [17]:
buildings_sections_gdf.head()

,reference,informatio,currentUse,AREA,MEAN,MWh_aprove,MWh_prod_e,geometry,centroid,Secciones,...,Total_poblacion,Porcentaje_Hogares_unipersonales,Porcentaje_mayor_65,Porcentaje_menor_18,Porcentaje_poblacion,Tamaño_medio_hogar,Municipios,area_m2,longitude,latitude
0,000100100XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,286.25,1487.646565,425.838829,76.650989,"MULTIPOLYGON (((690850.720 4229633.520, 690849...",POINT (-0.82064 38.19439),305905002,...,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07,-0.820638,38.194394
1,000100200XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,318.75,1377.799223,439.173502,79.051230,"MULTIPOLYGON (((690450.583 4229439.105, 690451...",POINT (-0.82509 38.19286),305905002,...,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07,-0.825093,38.192859
2,000100600XH92G,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,429.75,1466.006952,630.016488,113.402968,"MULTIPOLYGON (((690403.571 4229635.267, 690407...",POINT (-0.82580 38.19448),305905002,...,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07,-0.825796,38.194476
3,000200100XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,45.00,1456.424836,65.539118,11.797041,"MULTIPOLYGON (((691877.345 4226831.294, 691869...",POINT (-0.80969 38.16901),305905002,...,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07,-0.809692,38.169006
4,000200200XH92E,https://www1.sedecatastro.gob.es/CYCBienInmueb...,residential,113.25,1436.972393,162.737124,29.292682,"MULTIPOLYGON (((691878.565 4226850.104, 691880...",POINT (-0.80963 38.16908),305905002,...,8700,25.9,22.6,16.0,68.9,2.74,Crevillent,2.383811e+07,-0.809627,38.169083


In [19]:
buildings_coords_gdf = buildings_sections_gdf.drop(columns=['centroid']).copy()

In [20]:
buildings_coords_gdf.to_csv('buildings_sections.csv', index=False)